In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
######################

user_name='mohabu'
# user_name='Mhdella'

In [2]:
plot_clear_fg=0 ## No plots 
## plot_clear_fg=1
############

Exclud_port=1 ## Excluding time at the port and the fuel as well
# Exclud_port=0 ## Including time at the port and the fuel as well
###########

# ock_gr_fg=1; ## Choose Routes from Ockero to Groto
ock_gr_fg=0; ## Choose Routes from Groto to Ockero

rt_tp_cases_fg=1   ## Routes with top cases of eff. score
# rt_tp_cases_fg=0  ## Routes with bottom cases of eff. score 

#     ock_gr_fg=1; ## Choose Routes from Ockero to Groto
#     # ock_gr_fg=0; ## Choose Routes from Groto to Ockero

# rt_base=1 ## Base case for routes 
rt_base=0  ## This optio for top or bottom cases 

#     rt_tp_cases_fg=1   ## Routes with top cases of eff. score
#     # rt_tp_cases_fg=0  ## Routes with bottom cases of eff. score

##########
# rt_tp=1 ## 1st Top Route 
# rt_tp=2 ## 2nd Top Route 
# rt_tp=3 ## 3rd Top Route 

#     rt_bt=1 ## 1st Bottom Route 
#     rt_bt=2 ## 2nd Bottom Route 
#     rt_bt=3 ## 3rd Bottom Route 

###############################################################
time_sogfx=1; tim_tsh=10  #Limiting time, when the stepwise calculated distance is very small led to big time step 
# time_sogfx=0  #Not limiting time
##############

# time_limit_fg=1 ##time constriant condition1
# time_limit_fg=2 ##time constriant condition1
time_limit_fg=3 ##time constriant condition for top eff_score routes
#############

Phi_course_fg=0 #Using course deviations *Vessel's course direction with resepect to True North
#     Phi_course_fg=1 #using Phi as vessel's direction with respect to vessel's heading
#############

# delt_chg_fg=1  ## change sog, cog by 1 std of sog and cog of Target route
# delt_chg_fg=2  ## change sog, cog by 1/2 
# delt_chg_fg=3  ## change sog, cog by 1/4 
delt_chg_fg=4  ## change sog, cog by 1/8 

###############
course_fg=1  # vessel's course direction is as in the route data
# course_fg=0  # vessel's course direction is predicted from the sog model 
###########

past_fg=0; ##inputs = present measured values
# past_fg=1; ##inputs =  past values
###############

fl_meas_pred=1  #calculate the fuel for the RT from prediction (not measurements) to avoid uncertainty between the predicted optmized fuel and fuel RT
# fl_meas_pred=0  # Fuel RT from measurements
###############
max_norm=1  #normalize
# max_norm=0  #not normalize
################

# df_merg='April' # using April2020 data
# df_merg='April_Oct' # using April and October 2020 data
df_merg='All2020' # using April and October 2020 data

##############
# temp_fg=1  #to resample the target route  data
temp_fg=0  #Not to resample the target route  data
##############

############# Tempo_Simu  ### Simulation Results ##############

### tempo_simu='1S'

# tempo_simu='10S'

# tempo_simu='1min'

#############  Tempo_res  ### Route Results ##############

# tempo_res='1S'

# tempo_res='10S'

# tempo_res='30S'

tempo_res='1min'

# tempo_res='H'



In [3]:
import os
import pandas as pd
import numpy as np
import pickle
from datetime import timedelta

############################
# if ock_gr_fg==1:
#     df_ock_gr=pd.read_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/Sensitivity_Analysis/output/Eff_Sorted_datav3_0_norm_Rts_Ockero_Groto.xlsx')
# if ock_gr_fg==0:
#     df_ock_gr=pd.read_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/Sensitivity_Analysis/output/Eff_Sorted_datav3_0_norm_Rts_Groto_Ockero.xlsx')
    
# if ock_gr_fg==1:
#     df_ock_gr=pd.read_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/Sensitivity_Analysis/output/Sog_interv_Eff_Sorted_datav3_0_norm_Rts_Ockero_Groto.xlsx')
# if ock_gr_fg==0:
#     df_ock_gr=pd.read_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/Sensitivity_Analysis/output/Sog_interv_Eff_Sorted_datav3_0_norm_Rts_Groto_Ockero.xlsx')

if ock_gr_fg==1:
    df_ock_gr=pd.read_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/Data_v3/Scored_Rts_data2020/Eff_Sorted_datav3_1_norm_Rts_Ockero_Groto.xlsx')
if ock_gr_fg==0:
    df_ock_gr=pd.read_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/Data_v3/Scored_Rts_data2020/Eff_Sorted_datav3_1_norm_Rts_Groto_Ockero.xlsx')
        


In [4]:
df_ock_gr
len(df_ock_gr)

248

In [5]:
# (rt_tp_cases_fg==1) & (rt_base==0)
df_ock_gr.date_start.iloc[1]

Timestamp('2020-02-16 21:37:00')

In [6]:
kk=7
# st=stop

In [ ]:
dfst_start=[]; dfst_end=[]; dfst_time_ExPorts_all=[]
dfst_fuel_sel=[]; dfst_fuel_meas=[]; dfst_fuel_fx=[]
dfst_time_sel=[]; dfst_time_meas=[];  dfst_time_fx=[]
dfst_distance_sel=[]; dfst_distance_meas=[]; dfst_distance_fx=[]
dfst_improve_sel=[]; dfst_improve_meas=[]; dfst_improve_fx=[]; dfst_RTid=[]

dfst_combs_dist=pd.DataFrame([])
df_info=[]

df_RT_Stats_improve=pd.DataFrame({'start_time':[], 'end_time':[], 'Time_ExPorts':[],
                                 'fuel_sel':[], 'fuel_meas':[], 'fuel_fx':[],
                                 'time_sel':[], 'time_meas':[], 'time_fx':[],
                                'distance_sel':[], 'distance_meas':[],  'distance_fx':[],
                                 'improve_sel':[], 'improve_meas':[], 'improve_fx':[]})


for kk in range(len(df_ock_gr)):
# for i in range(1):

    rt_tp=kk ## The Chosen Top Route 
    rt_bt=kk ## The Chosen Bottom Route

    import os
    import pandas as pd
    import numpy as np
    import pickle
    from datetime import timedelta

    from bokeh.plotting import figure
    from bokeh.tile_providers import get_provider, Vendors, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
    from bokeh.io import output_notebook, show, output_file
    from bokeh.plotting import figure, ColumnDataSource
    from bokeh.palettes import PRGn, RdYlGn
    from bokeh.transform import linear_cmap,factor_cmap
    from bokeh.layouts import row, column
    from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
    import matplotlib.pyplot as plt


    import warnings
    warnings.filterwarnings("ignore")

    import numpy as np
    from sklearn.tree import DecisionTreeRegressor
    from sklearn import tree
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    from scipy.stats import linregress
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_absolute_error


    # In[4]:


    # df.shape
    tempo_res


    # ### The Selected Route
    # From Ockero to Groto
    # 
    # or From Groto to Ockero

    # In[5]:



    # In[6]:


    df_ock_gr.shape


    # In[7]:


    df=pd.read_pickle('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad\ECOKIT_MATERIAL/data_v3_1min.pkl')

    if (ock_gr_fg==1) & (rt_base==1):
        df_Rtrg=df[(df.index >= '2020-04-05 15:10:00') & (df.index < '2020-04-05 15:39:00')] ##from Ockero to Groto
    if (ock_gr_fg==0) & (rt_base==1):
        df_Rtrg=df[(df.index >= '2020-04-05 20:38:00') & (df.index <= '2020-04-05 21:07:00')] ##from Groto to Ockero

    if (rt_tp_cases_fg==1) & (rt_base==0):
        rt=rt_tp
        df_Rtrg=df[(df.index >= df_ock_gr.date_start.iloc[rt]) & (df.index <= df_ock_gr.date_end.iloc[rt])] 

    if (rt_tp_cases_fg==0) & (rt_base==0):
        rt=-rt_bt
        df_Rtrg=df[(df.index >= df_ock_gr.date_start.iloc[rt]) & (df.index <= df_ock_gr.date_end.iloc[rt])] 


    df_Rtrg.shape


    # In[8]:


    # df_ock_gr.date_start
    df_ock_gr.date_start.iloc[3]


    # In[9]:


    df_Rtrg.index


    # In[10]:


    # st1=stop


    # In[11]:


    # df_Rtrg[['latitude', 'longitude']]


    # In[12]:


    sog_max_limit=df.speedOverGround.max()


    # In[13]:


    df_Rtrg.head()
    # df_Rtrg.tail()


    # In[14]:


    # df_Raw_RT=df_Rtrg
    df_Raw_RT_orig=df_Rtrg
    df_Rtrg.columns


    # In[15]:


    df_Rtrg['directionTrue']=df_Rtrg['windDirectionTrue']*180/np.pi
    df_Rtrg['courseOverGroundTrue']=df_Rtrg['courseOverGroundTrue']*180/np.pi
    df_Rtrg['EngineFuelRate']=df_Rtrg['engineFuelRate']


    dfw = pd.DataFrame({"Wsp":df_Rtrg['windSpeedOverGround'],
                       "Vb":df_Rtrg['speedOverGround'],
                       "Wdr":df_Rtrg['directionTrue']/180*np.pi,
                       "Bdr":df_Rtrg['courseOverGroundTrue']/180*np.pi})


    dfw['alpha']=dfw.Bdr-dfw.Wdr
    dfw['Adiff']=dfw['alpha']
    dfw['cal_appsp']=np.sqrt(dfw.Wsp**2+dfw.Vb**2+2*dfw.Wsp*dfw.Vb*np.cos(dfw.Adiff))
    dfw['cal_appdr']=np.arccos((dfw.Wsp*np.cos(dfw.Adiff)+dfw.Vb)/(dfw.cal_appsp))


    df_Rtrg['speedApparent']=dfw['cal_appsp']
    df_Rtrg['speedApparent'].max()


    # In[16]:


    df_Rtrg['wind_course_diff']=df_Rtrg['directionTrue']-df_Rtrg['courseOverGroundTrue']

    df_Rtrg['wind_course_diff'].describe()
    # df_Rtrg['wind_head_diff'].describe()


    # In[17]:


    if tempo_res=='1min':
        df=df_Rtrg

    # if temp_fg==1:
    #     if day_cases==1:
    #         df=df_Rtrg
    #         ##df['dt']=pd.to_datetime((df['timestamp']), format='%Y-%m-%d %H:%M:%S')
    #         df['dt']=pd.to_datetime((df['timestamp']), format='%y/%m/%d-%H:%M:%S')
    #         df['date'] = df['dt']
    #         df = df.set_index(df['date'])
    #         # # df.drop('Unnamed: 0', axis=1, inplace=True)
    #         df=df.resample(tempo_res).mean()
    #         # df = df.dropna()
    #         df=df.reset_index()
    #         df.drop(['date', 'Unnamed: 0'], axis=1, inplace=True)

    if temp_fg==1:
        df=df_Rtrg
        df=df.resample(tempo_res).mean()
        if df.isna().sum().sum()!=0:
            df_filled=df.interpolate(method='linear', limit_direction='forward', axis=0)
            df=df_filled
    #         df = df.dropna()


    df.shape


    # In[18]:


    df_Raw_RT=df


    # In[19]:


    # df[['speedApparent', 'EngineFuelRate']].describe()


    # In[20]:


    df.columns


    # In[21]:


    # df_RT=df_Rtrg[['longitude', 'latitude', 'speedOverGround', 'windSpeedOverGround', 'directionTrue', 'courseOverGroundTrue', 'speedApparent','wind_course_diff', 'EngineFuelRate' ]]

    # df_RT=df[['longitude', 'latitude', 'speedOverGround', 'windSpeedOverGround', 'directionTrue', 'courseOverGroundTrue', 'speedApparent','wind_course_diff', 'EngineFuelRate', 'pitch' ]]

    # df_RT=df[['longitude', 'latitude', 'speedOverGround', 'windSpeedOverGround', 'directionTrue', 'courseOverGroundTrue', 'speedApparent',
    #           'wind_course_diff', 'EngineFuelRate', 'pitch', 'speedOverGround','speedApparent',
    #           'distanceFromDock', 'distanceToNextDock', 'distanceToPreviousDock','accumDistLeg','deltaDistance', 'accumTimeLeg']]

    df_RT=df

    df_RT.head()


    # In[22]:


    df_RT_orig=df_RT
    df_RT_sog_orig=df_RT['speedOverGround']
    df_RT_cogt_orig=df_RT['courseOverGroundTrue']
    df_RT.shape


    # In[23]:


    outputs=['EngineFuelRate']


    # outputs=['speedOverGround']

    # outputs=['speedOverGround','courseOverGroundTrue']

    # inputs=['windSpeedOverGround','speedOverGround','speedApparent']

    # inputs=['windSpeedOverGround','speedOverGround','speedApparent', 'wind_course_diff']

    # inputs=['speedOverGround','courseOverGroundTrue','distanceFromDock','wind_course_diff']

    # inputs=['distanceFromDock','wind_course_diff']

    # inputs=['speedOverGround','courseOverGroundTrue', 'windSpeedOverGround','directionTrue','distanceFromDock','accumDistLeg','accumTimeLeg',
    #        'distanceToNextDock', 'speedApparent', 'wind_course_diff'] 


    # inputs=['speedOverGround','courseOverGroundTrue', 'windSpeedOverGround', 'directionTrue','distanceFromDock','accumDistLeg','accumTimeLeg',
    #        'distanceToNextDock', 'speedApparent', 'pitch']  ### inputs of 60% model with rt_case=2

    inputs=['speedOverGround','courseOverGroundTrue', 'windSpeedOverGround', 'directionTrue','distanceFromDock','accumDistLeg','accumTimeLeg',
           'distanceToNextDock', 'speedApparent', 'pitch', 'wind_course_diff'] ###18improv_cs2, with fl_mod...score_0.7504.hdf5



    # In[24]:


    df_orig=df
    df_outs=df_orig[outputs]
    df_outs.head()


    # In[25]:


    if Phi_course_fg==0:
        df['courseOverGroundTrue'] = df['courseOverGroundTrue'] 
    if Phi_course_fg==1:
        df_course_orig=df['courseOverGroundTrue']
        df_heading_orig=df['headingMagnetic']*180/np.pi
        df['courseOverGroundTrue'] = df['courseOverGroundTrue'] - df['headingMagnetic']*180/np.pi


    # In[26]:


    df['headingMagnetic'].describe()
    df['courseOverGroundTrue'].describe()


    # In[27]:


    past_inputs=inputs


    # In[28]:


    # df = df.dropna()
    if past_fg==0:
        df1=df[past_inputs]
        temp=df[past_inputs].iloc[0]
        df1[past_inputs]=df[past_inputs]
        df1.iloc[0,:]=temp

        df[past_inputs]=df1[past_inputs]
        df[past_inputs].head()

    if past_fg==1:
        df1=df[past_inputs]
        temp=df[past_inputs].iloc[0]
        df1[past_inputs]=df[past_inputs].shift(+1)
        df1.iloc[0,:]=temp

        df[past_inputs]=df1[past_inputs]
        df[past_inputs].head()


    # In[29]:


    df_inps=df[past_inputs]


    # ### Data preprocessing for change in vessel's speed and course 

    # In[30]:


    df_ins=df[inputs]

    df_outs.shape
    df_ins.shape

    df_ins.isna().sum()
    df_outs.isna().sum()
    df_ins.head()
    df_outs.head()


    # In[31]:


    df_ins.head()


    # In[32]:


    df_inps.describe()
    mean=df_inps.describe().loc['mean']
    std=df_inps.describe().loc['std']
    min=df_inps.describe().loc['min']
    max=df_inps.describe().loc['max']
    Q1=df_inps.describe().loc['25%']
    std


    # In[33]:


    # st=stop
    # 
    # df_inps.accumDistLeg.iloc[12]/df_inps.speedOverGround.iloc[12]
    # df_inps.accumDistLeg
    # df_inps.iloc[12]
    # df_inps.speedOverGround.iloc[12]=0


    # In[34]:


    # df_inps.mean(axis=0)
    # df_inps.std(axis=0)


    # In[35]:


    # st=stop


    # In[36]:


    if delt_chg_fg==1:
        df_inc_s1=df_inps.add(df_inps.std(axis=0), axis=1)
        df_dec_s1=df_inps.sub(df_inps.std(axis=0), axis=1)
    if delt_chg_fg==2:
        df_inc_s1=df_inps.add(df_inps.std(axis=0)/2, axis=1)
        df_dec_s1=df_inps.sub(df_inps.std(axis=0)/2, axis=1)
    if delt_chg_fg==3:
        df_inc_s1=df_inps.add(df_inps.std(axis=0)/4, axis=1)
        df_dec_s1=df_inps.sub(df_inps.std(axis=0)/4, axis=1)
    if delt_chg_fg==4:
        df_inc_s1=df_inps.add(df_inps.std(axis=0)/8, axis=1)
        df_dec_s1=df_inps.sub(df_inps.std(axis=0)/8, axis=1)

    # df_inc_s1=df_inps.add(std, axis=1).head()
    # df_dec_s1=df_inps.sub(std, axis=1).head()

    # df_inc_s1


    # In[37]:


    ###### Operational Conditions
    df_dec_s1.speedOverGround[df_dec_s1.speedOverGround<0]=0
    df_inc_s1.speedOverGround[df_inc_s1.speedOverGround>sog_max_limit]=sog_max_limit
    df_inc_s1.courseOverGroundTrue[df_inc_s1.courseOverGroundTrue>360]=df_inc_s1.courseOverGroundTrue-360
    df_dec_s1.courseOverGroundTrue[df_dec_s1.courseOverGroundTrue<0]=df_dec_s1.courseOverGroundTrue+360
    # df_inc_s1.speedOverGround.describe()
    # df_dec_s1.speedOverGround.describe()
    # df_inc_s1.courseOverGroundTrue.describe()
    df_dec_s1.courseOverGroundTrue.describe()


    # In[38]:


    df_upsog_fxcog_s1=pd.DataFrame({'speedOverGround':df_inc_s1.speedOverGround, 'courseOverGroundTrue':df_ins.courseOverGroundTrue})
    df_upsog_upcog_s1=pd.DataFrame({'speedOverGround':df_inc_s1.speedOverGround, 'courseOverGroundTrue':df_inc_s1.courseOverGroundTrue})
    df_upsog_dncog_s1=pd.DataFrame({'speedOverGround':df_inc_s1.speedOverGround, 'courseOverGroundTrue':df_dec_s1.courseOverGroundTrue})

    df_dnsog_fxcog_s1=pd.DataFrame({'speedOverGround':df_dec_s1.speedOverGround, 'courseOverGroundTrue':df_ins.courseOverGroundTrue})
    df_dnsog_upcog_s1=pd.DataFrame({'speedOverGround':df_dec_s1.speedOverGround, 'courseOverGroundTrue':df_inc_s1.courseOverGroundTrue})
    df_dnsog_dncog_s1=pd.DataFrame({'speedOverGround':df_dec_s1.speedOverGround, 'courseOverGroundTrue':df_dec_s1.courseOverGroundTrue})

    df_fxsog_fxcog_s1=pd.DataFrame({'speedOverGround':df_ins.speedOverGround, 'courseOverGroundTrue':df_ins.courseOverGroundTrue})
    df_fxsog_upcog_s1=pd.DataFrame({'speedOverGround':df_ins.speedOverGround, 'courseOverGroundTrue':df_inc_s1.courseOverGroundTrue})
    df_fxsog_dncog_s1=pd.DataFrame({'speedOverGround':df_ins.speedOverGround, 'courseOverGroundTrue':df_dec_s1.courseOverGroundTrue})


    # In[39]:


    # inp_noincr=['windSpeedOverGround', 'directionTrue','distanceFromDock', 'accumDistLeg', 'accumTimeLeg','distanceToNextDock']
    inp_noincr=['windSpeedOverGround', 'directionTrue','distanceFromDock', 'accumDistLeg', 'accumTimeLeg','distanceToNextDock','pitch']

    # df_inc_s1[inp_noincr]=df_ins[inp_noincr]
    # df_dec_s1[inp_noincr]=df_ins[inp_noincr]


    # In[40]:


    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']    
    for i in sog:
        for j in cog:
            eval(i+'_'+j)[inp_noincr]=df_ins[inp_noincr]


    # In[41]:


    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']    
    for i in sog:
        for j in cog:
            dfw = pd.DataFrame({"Wsp":df_ins['windSpeedOverGround'],
                           "Vb":eval(i+'_'+j)['speedOverGround'],
                           "Wdr":df_ins['directionTrue']/180*np.pi,
                           "Bdr":eval(i+'_'+j)['courseOverGroundTrue']/180*np.pi})


            dfw['alpha']=dfw.Bdr-dfw.Wdr
            dfw['Adiff']=dfw['alpha']
            dfw['cal_appsp']=np.sqrt(dfw.Wsp**2+dfw.Vb**2+2*dfw.Wsp*dfw.Vb*np.cos(dfw.Adiff))
            dfw['cal_appdr']=np.arccos((dfw.Wsp*np.cos(dfw.Adiff)+dfw.Vb)/(dfw.cal_appsp))


            eval(i+'_'+j)['speedApparent']=dfw['cal_appsp']
            eval(i+'_'+j)['speedApparent']=eval(i+'_'+j)['speedApparent']
#             print('speedApparent', eval(i+'_'+j)['speedApparent'].max())

            eval(i+'_'+j)['wind_course_diff']=eval(i+'_'+j)['directionTrue']-eval(i+'_'+j)['courseOverGroundTrue']
#             print('wind_course_diff', eval(i+'_'+j)['wind_course_diff'].max())



    # In[42]:


    X={}
    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']    
    for i in sog:
        for j in cog:
            df_ins=eval(i+'_'+j)[inputs]
            df_ins.head()
            df_outs.head()

            if max_norm==1:
                df_ins_norm= df_ins/ df_ins.max(axis=0)
                # df_ins_norm = (df_ins - df_ins.min()) / (df_ins.max() - df_ins.min())

                df_outs_norm= df_outs/ df_outs.max(axis=0)
                # df_outs_norm = (df_outs - df_outs.min()) / (df_outs.max() - df_outs.min())

                df_ins=df_ins_norm
                df_outs=df_outs_norm
                X[i+'_'+j] = df_ins
                y = df_outs


    # In[43]:


    df_ins.head()


    # In[44]:


    X
    X['df_upsog_fxcog_s1'].head()


    # In[45]:


    X['df_fxsog_fxcog_s1'].head()


    # In[46]:


    y.shape


    # ### ANN modeling

    # In[47]:


    import os
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    # from sklearn.externals import joblib
    import joblib
    from sklearn.model_selection import train_test_split
    from keras.models import Sequential
    from keras.layers import Dense, LSTM
    from keras import backend as k
    import keras
    from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
    from sklearn.metrics import r2_score, mean_squared_error
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    import numpy as np
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    from scipy.stats import linregress
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score


    # In[48]:


    # Define r2_keras as an accuracy metrics
    def r2_keras(y_true, y_pred):

        ss_res = k.sum(k.square(y_true - y_pred))
        ss_tot = k.sum(k.square(y_true - k.mean(y_true)))
        return 1-ss_res/(ss_tot + k.epsilon())

    # Define rmse as a loss function
    def rmse(y_true, y_pred):

        return k.sqrt(k.mean(k.square(y_true - y_pred)))


    # In[49]:


    # X.shape
    y.shape      


    # ### Predict Fuel from changes of inpouts

    # In[50]:


    # model_path = 'C:/Users/'+user_name+'/iHelm/ECO KIT/Data_analysi_final/'
    model_path = 'C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/ECO KIT/Data_analysi_final/'


    from keras.models import load_model
    # sog_model = load_model(model_path+'mod_1min_outs_1_lyrs_3_epc_1000_Neo_25.hdf5', custom_objects={'rmse':rmse, 'r2_keras':r2_keras})

    # sog_model = load_model(model_path+'sog_mod_outs_speedOverGround_ins_windSpeedOverGround speedOverGround speedApparent wind_course_diff_1min_outs_1_lyrs_3_epc_500_Neo_25_score_0.40284936145903705.hdf5',
    #                        custom_objects={'rmse':rmse, 'r2_keras':r2_keras})



    #############################

    if max_norm==1:
    #     fl_model = load_model(model_path+'Fl_pred_Otssog_cogt_ins_sog_cogt_wsog_directionTrue_dsFromdk_accumDistLeg_accumTimeLeg_dsTndk_spap_wcdf_norm_1_tpfl_smps_0_1min_outs_1_lyrs_3_epc_1000_Neo_25_score_0.6017.hdf5',
    #                         custom_objects={'rmse':rmse, 'r2_keras':r2_keras})
    ####     fl_model = load_model(model_path+'fl_mod_outs_fl_ins_sog_cog_wsog_dr_dsFromdk_accumDistLeg_accumTimeLeg_dstpdk_spApparent_pitch_norm_1_tpfl_smps_1_1min_outs_1_lyrs_3_epc_1000_Neo_25_score_0.7371.hdf5',
    #                         custom_objects={'rmse':rmse, 'r2_keras':r2_keras})

    #     fl_model = load_model(model_path+'fl_mod_outs_fl_ins_sog_cog_wsog_dr_dsFromdk_accumDistLeg_accumTimeLeg_dstpdk_spApparent_pitch_norm_1_tpfl_smps_1_1min_outs_1_lyrs_3_epc_1000_Neo_25_score_0.7333.hdf5',
    #                         custom_objects={'rmse':rmse, 'r2_keras':r2_keras})### gives 60% for rt_case=2, inputs no wind_diff

    #     fl_model = load_model(model_path+'fl_mod_outs_fl_ins_sog_cog_wsog_dr_dsFromdk_adl_accumTimeLeg_dstpdk_spApparent_pitch_wind_course_diff_norm_1_tpfl_smps_1_1min_outs_1_lyrs_3_epc_1000_Neo_25_score_0.7504.hdf5',
    #                         custom_objects={'rmse':rmse, 'r2_keras':r2_keras})  ###18improv_cs2

    #     if Phi_course_fg==1:
        if (Phi_course_fg==1) | (Phi_course_fg==0):
            if user_name=='Mhdella':
                fl_model = load_model(model_path+'fl_mod_outs_fl_ins_sog_cog_wsog_dr_dsFromdk_adl_accumTimeLeg_dstpdk_spApparent_pitch_wind_course_diff_phi_1_norm_1_tpfl_smps_1_1min_outs_1_lyrs_3_epc_1000_Neo_25_score_0.7457.hdf5',
                            custom_objects={'rmse':rmse, 'r2_keras':r2_keras})  ### with course with respect to heading
            if user_name=='mohabu':
                fl_model = load_model(model_path+'fl_mod_outs_fl_ins_sog_cog_wsog_dr_dsFromdk_adl_accumTimeLeg_dstpdk_spAppt_ptch_wd_crse_diff_phi_1_norm_1_tpfl_smps_1_1min_outs_1_lyrs_3_epc_1000_Neo_25_score_0.7457.hdf5',
                            custom_objects={'rmse':rmse, 'r2_keras':r2_keras})  ### with course with respect to heading

    #         if tempo_res=='30S':
    #             fl_model = load_model(model_path+'fl_mod_outs_fl_ins_sog_cog_wsog_dr_dsFromdk_adl_accumTimeLeg_dstpdk_spApparent_pitch_wcd_phi_1_norm_1_tpfl_smps_1_30S_outs_1_lyrs_3_epc_1000_Neo_25_score_0.707.hdf5',
    #                         custom_objects={'rmse':rmse, 'r2_keras':r2_keras})  ### with course with respect to heading



    # In[51]:


    # fl_model.summary()
    # score = fl_model.evaluate(X, y, verbose=0)
    # print("%s: %.2f%%" % (fl_model.metrics_names[1], score[1]*100))


    # In[52]:


    y_pred={}
    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']    
    for i in sog:
        for j in cog:
            X_test=X[i+'_'+j]
            if X_test.isna().sum().sum()!=0:
                X_test_filled=X_test.interpolate(method='linear', limit_direction='forward', axis=0)
                X_test=X_test_filled
            Y_test=y
            y_pred_ANN = fl_model.predict(x=X_test, batch_size=None, verbose=0, steps=None)
            y_pred_ANN[:,0][np.isnan(y_pred_ANN[:,0])] = 0
            y_pred[i+'_'+j]=y_pred_ANN[:,0]


    # In[53]:


    X_test.shape
    # Y_test.shape


    # In[54]:


    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']  
    for i in sog:
        for j in cog:
            Y_pred=y_pred[i+'_'+j]
#             print('R2 Testing data Fuel=', r2_score(Y_test.iloc[:,0].values, Y_pred))
            # print('R2 Testing data speed=', r2_score(Y_test[:,1], y_pred[:,1]))
#             print('Cycle total Testing data=', 100*(1-(abs(sum(Y_pred)-  Y_test.iloc[:,0].sum()) /  Y_test.sum())))
#             print('RMSE Testing data Fuel=', np.sqrt(mean_squared_error(Y_test.iloc[:,0], Y_pred)))


    # In[55]:


    X_test.iloc[:,0].max()


    # In[56]:


    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1'] 
    
    if plot_clear_fg==1:
        import matplotlib.pyplot as plt
        x_ax = range(len(Y_test))
        plt.figure(figsize=(12, 6))
        plt.plot(x_ax, Y_test.iloc[:,0]*df.EngineFuelRate.max(), label="Measured")
        for i in sog:
            for j in cog:
                Y_pred=y_pred[i+'_'+j]
                plt.plot(x_ax, Y_pred*df.EngineFuelRate.max(), label=i+'_'+j)

        plt.title(outputs[0]+'test and predicted data')
        plt.xlabel('Time_steps')
        plt.ylabel(outputs[0])
        plt.legend(bbox_to_anchor = (1, 1))
        plt.grid(True)
        plt.show() 


    # In[57]:


    # sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    # cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']  
    # for i in sog:
    #     for j in cog:
    #         Y_pred=y_pred[i+'_'+j]
    #         import matplotlib.pyplot as plt
    #         x_ax = range(len(Y_test))
    #         plt.figure(figsize=(12, 6))
    #         plt.plot(x_ax, Y_test.iloc[:,0]*df.EngineFuelRate.max(), label="Measured")
    #         plt.plot(x_ax, Y_pred*df.EngineFuelRate.max(), label=i+'_'+j)
    #         plt.title(outputs[0]+'test and predicted data')
    #         plt.xlabel('Time_steps')
    #         plt.ylabel(outputs[0])
    #         plt.legend(bbox_to_anchor = (1, 1))
    #         plt.grid(True)
    #         plt.show()  


    # ### For Presentation

    # In[58]:


    # df.columns.tolist()
    df.shape


    # In[59]:


    df.EngineFuelRate.head()


    # In[60]:


    df_fl=pd.DataFrame({'Meas_fuel':df.EngineFuelRate.values})

    if max_norm==0:
        df_Rt_Res=pd.concat([df_fl, pd.DataFrame.from_dict(y_pred)], axis=1)

    if max_norm==1:
        df_Rt_Res=pd.concat([df_fl, pd.DataFrame.from_dict(y_pred)*df.EngineFuelRate.max()], axis=1)

    df_Rt_Res.head()


    # In[61]:


    tempo_res


    # In[62]:


    dfcor=df_Rt_Res
    #######################

    R_f={}; R_fname={}
    R=0

    fuelcns=[]
    df_all=pd.DataFrame([])

    if fl_meas_pred==0:
        fuel_vars=list(df_Rt_Res.columns)
    if fl_meas_pred==1:
        fuel_vars=list(df_Rt_Res.columns)

    for F in fuel_vars:
        dfcor['dfr1']=dfcor[F]
        dfcor['dfr2']=dfcor[F].shift(-1)

        dfcor = dfcor.reset_index(drop=True)

        #     if tempo_res=='1min':
        #         d_feulcns=abs((dfcor['dfr2']-dfcor['dfr1'])/(60))
        #     if tempo_res=='1S':
        #         d_feulcns=abs((dfcor['dfr2']-dfcor['dfr1'])/(60*60))
        #     if tempo_res=='10S':
        #         d_feulcns=abs((dfcor['dfr2']-dfcor['dfr1'])/(60*10))

        if tempo_res=='1min':
            d_feulcns=dfcor['dfr1']/(60)
        if tempo_res=='1S':
            d_feulcns=dfcor['dfr1']/(60*60)
        if tempo_res=='10S':
            d_feulcns=dfcor['dfr1']/(60*60)*10
        if tempo_res=='30S':
            d_feulcns=dfcor['dfr1']/(60*60)*30


        R_f[F]=d_feulcns.sum()


        df_all=pd.concat([df_all, dfcor])

        df_R=pd.DataFrame({ 'RT_Fuel':R_f})


    # In[63]:


    df_R=df_R.T


    # In[64]:


    df_R


    # In[65]:


    df_R['date']=df.index.map(lambda t: t.strftime('%Y-%m-%d %H:%M'))[0]
    df_R=df_R.set_index(['date'])
    # del df_R['date']

    df_R=df_R.rename(columns = {'EngineFuelRate':'Engine Fuel'})

    df_R


    # In[66]:


    df_Rrank=df_R

    # df_sim_dist_fuel=df_Rrank

    df_sim_dist_fuel = df_Rrank.sort_values(by = ['Meas_fuel'], ascending = False)


    df_sim_dist_fuel

    # df_sim_dist_fuel.to_excel("df_R_dataframe_April_1st_10similar_routes.xlsx")


    # In[67]:

    if plot_clear_fg==1:
        import matplotlib.pyplot as plt

        df_R = df_sim_dist_fuel

        # fig = plt.figure(figsize=(30,18))
        plt.rcParams["figure.figsize"] = (10, 5)


        df_R.plot.bar(rot=75, zorder=3)
        plt.title('Fuel', fontsize=18)
        plt.ylabel('Fuel Consumption (L)', fontsize=15, labelpad=10)
        plt.xlabel('', fontsize=15, labelpad=10)
        plt.yticks(fontsize=12);
        plt.xticks(fontsize=12, rotation = 0);
        plt.title('Fuel', fontsize=14) 
        plt.legend(bbox_to_anchor = (0.99, 1))
        plt.grid()



        plt.tight_layout()

        # fig.savefig('/content/gdrive/MyDrive/Investig_Hists_Routes/output2/Sorted_Hist_'+leg_name+'_'+day_start+'_'+day_end+'.png', dpi=100)

        plt.show();


    # In[68]:


    improve=(df_R['Meas_fuel'].values-df_R.values)/df_R['Meas_fuel'].values*100

    # print('improve of Pred fuel vs Meas_fuel',np.round(improve[0],2),'%')
    improvement=pd.DataFrame(data = improve[0]).T
    improvement.columns=df_R.columns
    improvement


    # In[69]:


    df_Rt_Res
    df_improv=df_R
    df_improv=df_improv.append(improvement)
    # df_improv.index.tolist()
    df_improv.index=['fuel consumption', 'improve over Meas_fuel']
    df_improv


    # ### Plot Routes for  Measured and Predicted Fuel

    # In[70]:


    df=df_Raw_RT
    # ##df['dt']=pd.to_datetime((df['timestamp']), format='%Y-%m-%d %H:%M:%S')
    # df['dt']=pd.to_datetime((df['timestamp']), format='%y/%m/%d-%H:%M:%S')
    # df['date'] = df['dt']
    # df = df.set_index(df['date'])
    # # df.drop('Unnamed: 0', axis=1, inplace=True)
    df=df.resample('1S').mean()
    # # df = df.dropna()
    # df=df.reset_index()
    # df.drop(['date', 'Unnamed: 0'], axis=1, inplace=True)

    df_filled1=df.interpolate(method='linear', limit_direction='forward', axis=0)
    df_filled2=df_filled1.interpolate(method='linear', limit_direction='backward', axis=0)

    df=df_filled2
    df.shape


    # In[71]:


    df.latitude.isna ().sum ()


    # In[72]:


    ###################################################
    # Define function to switch from lat/long to mercator coordinates
    def x_coord(x, y):

        lat = x
        lon = y

        r_major = 6378137.000
        x = r_major * np.radians(lon)
        scale = x/lon
        y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
            lat * (np.pi/180.0)/2.0)) * scale
        return (x, y)
    ###################################################
    output_notebook()

    # var2='EngineFuelRate'
    var2='speedOverGround'

    # var2='EngineSpeed'
    # var2='speedApparent'

    dfloc=df

    dfloc['coordinates'] = list(zip(dfloc['latitude'], dfloc['longitude']))

    # Obtain list of mercator coordinates
    mercators = [x_coord(x, y) for x, y in dfloc['coordinates']]
    # Create mercator column in our df
    dfloc['mercator'] = mercators

    # Split that column out into two separate cols - mercator_x and mercator_y
    dfloc[['mercator_x', 'mercator_y']] = dfloc['mercator'].apply(pd.Series)
    ####################################################

    # Select tile set to use
    chosentile = get_provider(Vendors.STAMEN_TERRAIN)
    # chosentile = get_provider(Vendors.WIKIMEDIA)
    chosentile = get_provider(WIKIMEDIA)

    # Choose palette
    # palette = PRGn[11]
    # palette = PRGn[8]
    from bokeh.palettes import Plasma256
    palette=Plasma256

    # Tell Bokeh to use df as the source of the data
    source = ColumnDataSource(data=dfloc)

    # Define color mapper - which column will define the colour of the data points
    # Define color mapper - which column will define the colour of the data points
    color_mapper = linear_cmap(field_name = var2, palette = palette, low = dfloc[var2].min(), high = dfloc[var2].max())
    color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                        formatter = NumeralTickFormatter(format='0.0[0000]'), 
                        label_standoff = 13, width=8, location=(0,0))

    q = figure(title ='Vessel Route: Grötö-Kalvsund-Framnäs-Öckero, Gothenburg, Sweden '+var2, x_axis_type="mercator", y_axis_type="mercator", 
                x_axis_label = 'Longitude', y_axis_label = 'Latitude')
    q.add_layout(color_bar, 'right')



    # Add map tile
    q.add_tile(chosentile)
      #Defines color bar    
    q.circle(x = 'mercator_x', y = 'mercator_y', line_color=color_mapper, color = color_mapper, source=source, size=2, fill_color= color_mapper, fill_alpha =0.7)

    q.add_tile(chosentile)



    # # #Save as HTML
    # output_file('/content/gdrive/MyDrive/Colab Notebooks/BokehAvocado-master/output/'+str(R_sz)+'_Sel_Routes_'+var2+'_Plot.html', title='Ferry Route in Gothenburg')


    if plot_clear_fg==1:
        show(q)



    # In[73]:


    print(df.shape)
    # df.columns


    # In[74]:


    dfloc.columns


    # In[75]:


    df_inc_s1=df.add(df.std(axis=0), axis=1)
    df_dec_s1=df.sub(df.std(axis=0), axis=1)

    ###### Operational Conditions
    df_dec_s1.speedOverGround[df_dec_s1.speedOverGround<0]=0
    df_inc_s1.speedOverGround[df_inc_s1.speedOverGround>sog_max_limit]=sog_max_limit
    df_inc_s1.speedOverGround.max()
    # df_dec_s1.speedOverGround.min()


    # In[76]:


    df.courseOverGroundTrue.std()


    # In[77]:


    if Phi_course_fg==0:
        dfloc['Phi_inc']=(df_inc_s1.courseOverGroundTrue-df.courseOverGroundTrue)
    #     abs(dfloc['Phi_inc'])

    if Phi_course_fg==1:
        dfloc['Phi_inc']=(df_inc_s1.courseOverGroundTrue-df.courseOverGroundTrue)
    #     dfloc['Phi_inc']=df_inc_s1.courseOverGroundTrue
        #     dfloc['Phi_inc']=(df_inc_s1.courseOverGroundTrue-df_course_orig)+df_heading_orig

    #     abs(dfloc['Phi_inc'])


    # ####  Beta is the angle of vessel's direction, which calculated based on the past latitude & longitude and next vessl's cog

    # In[78]:


    ###################################################
    # Define function to switch from lat/long to mercator coordinates
    import math

    def Beta(lat, lon):
    ### X = cos θb * sin ∆L
    ### Y = cos θa * sin θb – sin θa * cos θb * cos ∆L
    ### β = atan2(X,Y)
        Beta={}
        beta=0
        lat_a=np.radians(lat)
        lat_b=np.radians(lat.shift(-1).replace(np.nan,0))
        delta_lon=np.radians(lon.diff(1).shift(-1).replace(np.nan,0))

        X_loc= np.cos(lat_b) * np.sin(delta_lon)
        Y_loc= np.cos(lat_a) * np.sin(lat_b) - np.sin(lat_a) * np.cos(lat_b) * np.cos(delta_lon)
        for i in range(len(X_loc)):
            Beta[i]=math.atan2(X_loc[i], Y_loc[i])*180/np.pi

        return (Beta)
    ###################################################


    # In[79]:


    beta=Beta(dfloc.latitude, dfloc.longitude)

    dfloc['beta']=beta.values()
    #### dfloc.beta[dfloc.beta < 0]=dfloc.beta+360
    dfloc["beta"].describe()


    # In[80]:


    pd.set_option('display.max_rows', None)
    df_shw=dfloc[['speedOverGround','beta', 'latitude', 'longitude']]

    # df_shw[(df_shw.latitude>=57.696)&(df_shw.latitude<=57.7)]
    # df_shw[(df_shw.latitude>=57.6951)&(df_shw.latitude<=57.6953)]


    # In[81]:


    Phi_crse=dfloc['Phi_inc']/180*np.pi
    ######################################

    dfloc[['mercator_x', 'mercator_y']]

    dfcor=dfloc
    dfcor['lat1']=dfcor['mercator_y']
    dfcor['lat2']=dfcor['mercator_y'].shift(-1)
    dfcor['lon1']=dfcor['mercator_x']
    dfcor['lon2']=dfcor['mercator_x'].shift(-1)
    dfcor = dfcor.reset_index(drop=True)
    dfcor['lat2'].iloc[-1]=dfcor['lat2'].iloc[-2] 
    dfcor['lon2'].iloc[-1]=dfcor['lon2'].iloc[-2] 
    dlon = dfcor['lon2'] - dfcor['lon1']
    dlat = dfcor['lat2'] - dfcor['lat1']
    dist=np.sqrt(dlon**2+dlat**2)

    a = dist.values/np.cos(Phi_crse.astype(float))*np.sin(Phi_crse.astype(float))
    b = dist.values

    dfloc['mercator_x_inc']=dfloc.mercator_x+a
    dfloc['mercator_y_inc']=dfloc.mercator_y+b


    RT_dr=dfloc["beta"]

    bm=45
    # bm=67


    dfloc['mercator_x_inc'][(RT_dr <= bm) & (RT_dr > -bm)]=dfloc.mercator_x+a
    dfloc['mercator_y_inc'][(RT_dr <= bm) & (RT_dr > -bm)]=dfloc.mercator_y+b
    dfloc['mercator_x_inc'][(RT_dr >= 180-bm) & (RT_dr <= -90-bm)]=dfloc.mercator_x+a
    dfloc['mercator_y_inc'][(RT_dr >= 180-bm) & (RT_dr <= -90-bm)]=dfloc.mercator_y+b

    dfloc['mercator_x_inc'][(RT_dr > 90-bm) & (RT_dr < 180-bm)]=dfloc.mercator_x+b
    dfloc['mercator_y_inc'][(RT_dr > 90-bm) & (RT_dr < 180-bm)]=dfloc.mercator_y+a
    dfloc['mercator_x_inc'][(RT_dr > -180+bm) & (RT_dr < -bm)]=dfloc.mercator_x+b
    dfloc['mercator_y_inc'][(RT_dr > -180+bm) & (RT_dr < -bm)]=dfloc.mercator_y+a



    # dfloc[['mercator_x_inc', 'mercator_y_inc']].head()


    # In[82]:


    ###################################################
    # Define function to switch from lat/long to mercator coordinates
    import math

    def x_latlon(x, y):

        merc_x = x
        merc_y = y

        r_major = 6378137.000
        lon = merc_x/r_major*180/np.pi
        scale = x/lon
    #     y = 180/np.pi * np.log(np.tan(np.pi/4 + lat * (np.pi/180)/2)) * scale
    #     np.exp(y/scale*np.pi/180)=np.tan(np.pi/4 + lat * (np.pi/180)/2)
    #     math.atan(np.exp(y/scale*np.pi/180)) =np.pi/4 + lat * (np.pi/180)/2
        lat=(math.atan(np.exp(y/scale*np.pi/180))-np.pi/4)/((np.pi/180)/2) 

        return (lon, lat)
    ###################################################


    # In[83]:


    dfloc['latlon'] = list(zip(dfloc['mercator_x_inc'], dfloc['mercator_y_inc']))
    # Obtain list of lonlat coordinates
    latlons = [x_latlon(x, y) for x, y in dfloc['latlon']]
    # Create lanlot column in our df
    dfloc['lanlon'] = latlons
    # Split that column out into two separate cols - lat and lon
    dfloc[['Lon_inc', 'Lat_inc']] = dfloc['lanlon'].apply(pd.Series)
    # dfloc[['Lat_inc', 'Lon_inc']]


    # In[84]:


    if Phi_course_fg==0:
        dfloc['Phi_dec']=(df_dec_s1.courseOverGroundTrue-df.courseOverGroundTrue)

    if Phi_course_fg==1:
        dfloc['Phi_dec']=(df_dec_s1.courseOverGroundTrue-df.courseOverGroundTrue)
    #     dfloc['Phi_dec']=df_dec_s1.courseOverGroundTrue
    #     dfloc['Phi_dec']=(df_dec_s1.courseOverGroundTrue-df_course_orig)+df_heading_orig


    # In[85]:


    Phi_crse=dfloc['Phi_dec']/180*np.pi
    ### Phi_crse=abs(dfloc['Phi_dec'])/180*np.pi

    ######################################

    dfloc[['mercator_x', 'mercator_y']]

    dfcor=dfloc
    dfcor['lat1']=dfcor['mercator_y']
    dfcor['lat2']=dfcor['mercator_y'].shift(-1)
    dfcor['lon1']=dfcor['mercator_x']
    dfcor['lon2']=dfcor['mercator_x'].shift(-1)
    dfcor = dfcor.reset_index(drop=True)
    dfcor['lat2'].iloc[-1]=dfcor['lat2'].iloc[-2] 
    dfcor['lon2'].iloc[-1]=dfcor['lon2'].iloc[-2] 
    dlon = dfcor['lon2'] - dfcor['lon1']
    dlat = dfcor['lat2'] - dfcor['lat1']
    dist=np.sqrt(dlon**2+dlat**2)

    a = dist.values/np.cos(Phi_crse.astype(float))*np.sin(Phi_crse.astype(float))
    b = dist.values

    dfloc['mercator_x_dec']=dfloc.mercator_x+a
    dfloc['mercator_y_dec']=dfloc.mercator_y+b


    RT_dr=dfloc["beta"]

    bm=45
    # bm=67


    dfloc['mercator_x_dec'][(RT_dr <= bm) & (RT_dr > -bm)]=dfloc.mercator_x+a
    dfloc['mercator_y_dec'][(RT_dr <= bm) & (RT_dr > -bm)]=dfloc.mercator_y+b
    dfloc['mercator_x_dec'][(RT_dr >= 180-bm) & (RT_dr <= -90-bm)]=dfloc.mercator_x+a
    dfloc['mercator_y_dec'][(RT_dr >= 180-bm) & (RT_dr <= -90-bm)]=dfloc.mercator_y+b

    dfloc['mercator_x_dec'][(RT_dr > 90-bm) & (RT_dr < 180-bm)]=dfloc.mercator_x+b
    dfloc['mercator_y_dec'][(RT_dr > 90-bm) & (RT_dr < 180-bm)]=dfloc.mercator_y+a
    dfloc['mercator_x_dec'][(RT_dr > -180+bm) & (RT_dr < -bm)]=dfloc.mercator_x+b
    dfloc['mercator_y_dec'][(RT_dr > -180+bm) & (RT_dr < -bm)]=dfloc.mercator_y+a



    # dfloc[['mercator_x_dec', 'mercator_y_dec']].head()


    # In[86]:


    dfloc['latlon'] = list(zip(dfloc['mercator_x_dec'], dfloc['mercator_y_dec']))
    # Obtain list of lonlat coordinates
    latlons = [x_latlon(x, y) for x, y in dfloc['latlon']]
    # Create lanlot column in our df
    dfloc['lanlon'] = latlons
    # Split that column out into two separate cols - lat and lon
    dfloc[['Lon_dec', 'Lat_dec']] = dfloc['lanlon'].apply(pd.Series)
    dfloc[['Lat_dec', 'Lon_dec']].head()


    # ### Plot routes and limits

    # In[87]:


    # Define function to switch from lat/long to mercator coordinates
    def x_coord(x, y):

        lat = x
        lon = y

        r_major = 6378137.000
        x = r_major * np.radians(lon)
        scale = x/lon
        y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
            lat * (np.pi/180.0)/2.0)) * scale
        return (x, y)
    ###################################################
    output_notebook()

    # var2='engineFuelRate'
    var2='speedOverGround'

    dfloc=df

    for i in np.arange(3):
        if i==1:
            dfloc['latitude']=dfloc['Lat_inc']
            dfloc['longitude']=dfloc['Lon_inc']
        if i==2:
            dfloc['latitude']=dfloc['Lat_dec']
            dfloc['longitude']=dfloc['Lon_dec']

        dfloc['coordinates'] = list(zip(dfloc['latitude'], dfloc['longitude']))

        # Obtain list of mercator coordinates
        mercators = [x_coord(x, y) for x, y in dfloc['coordinates']]
        # Create mercator column in our df
        dfloc['mercator'] = mercators

        # Split that column out into two separate cols - mercator_x and mercator_y
        dfloc[['mercator_x', 'mercator_y']] = dfloc['mercator'].apply(pd.Series)

        ####################################################

        # Select tile set to use
    #     chosentile = get_provider(Vendors.STAMEN_TERRAIN)
        # chosentile = get_provider(Vendors.WIKIMEDIA)
        chosentile = get_provider(WIKIMEDIA)

        # Choose palette
        # palette = PRGn[11]
        # palette = PRGn[8]
        from bokeh.palettes import Plasma256
        palette=Plasma256

        # Tell Bokeh to use df as the source of the data
        source = ColumnDataSource(data=dfloc)

        # Define color mapper - which column will define the colour of the data points
        # Define color mapper - which column will define the colour of the data points
        color_mapper = linear_cmap(field_name = var2, palette = palette, low = dfloc[var2].min(), high = dfloc[var2].max())
        color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                            formatter = NumeralTickFormatter(format='0.0[0000]'), 
                            label_standoff = 13, width=8, location=(0,0))

        if i==0:
            q = figure(title ='Vessel Route: Grötö-Kalvsund-Framnäs-Öckero, Gothenburg, Sweden '+var2, x_axis_type="mercator", y_axis_type="mercator", 
                    x_axis_label = 'Longitude', y_axis_label = 'Latitude')
            q.add_layout(color_bar, 'right')
            q.circle(x = 'mercator_x', y = 'mercator_y', line_color=color_mapper, color = color_mapper, source=source, size=2, fill_color= color_mapper, fill_alpha =0.7)
        if i==1:
            ## Add map tile
            q.add_tile(chosentile)
            ##Defines color bar    
    #         q.circle(x = 'mercator_x', y = 'mercator_y', line_color=color_mapper, color = color_mapper, source=source, size=2, fill_color= color_mapper, fill_alpha =0.7)
    #         q.circle(x = 'mercator_x', y = 'mercator_y', line_color='red', color = 'red', source=source, size=1, fill_color= "r", fill_alpha =0.7)
            q.circle(x = 'mercator_x', y = 'mercator_y', line_color='grey', color = 'grey', source=source, size=1, fill_color= "r", fill_alpha =0.5)

        if i==2:
            ## Add map tile
            q.add_tile(chosentile)
            ##Defines color bar    
    #         q.circle(x = 'mercator_x', y = 'mercator_y', line_color=color_mapper, color = color_mapper, source=source, size=2, fill_color= color_mapper, fill_alpha =0.7)
    #         q.circle(x = 'mercator_x', y = 'mercator_y', line_color='green', color = 'green', source=source, size=1, fill_color= "green", fill_alpha =0.7)
            q.circle(x = 'mercator_x', y = 'mercator_y', line_color='grey', color = 'grey', source=source, size=1, fill_color= "green", fill_alpha =0.5)



        # output_file('/content/gdrive/MyDrive/Colab Notebooks/BokehAvocado-master/output/'+str(R_sz)+'_Sel_Routes_'+var2+'_Plot.html', title='Ferry Route in Gothenburg')


    if plot_clear_fg==1:
        show(q)



    # In[88]:


    pd.set_option('display.max_rows', None)
    # dfloc[['latitude', 'longitude']]
    # dfloc['courseOverGroundTrue']
    # dfloc['headingMagnetic']
    dfloc['heading']=dfloc['headingMagnetic']*180/np.pi
    # # dfloc[['courseOverGroundTrue','latitude', 'longitude', 'Lat_inc', 'Lon_inc', 'Lat_dec', 'Lon_dec']]
    # dfloc[['speedOverGround','courseOverGroundTrue','heading', 'latitude', 'longitude', 'Lat_inc', 'Lon_inc', 'Lat_dec', 'Lon_dec']].iloc[0:260]
    df_samp=dfloc[['speedOverGround','courseOverGroundTrue','heading','beta', 'latitude', 'longitude', 'Lat_inc', 'Lon_inc', 'Lat_dec', 'Lon_dec']]

    # df_samp[(df_samp.latitude>=57.696)&(df_samp.latitude<=57.699)]
    # df_samp[(df_samp.latitude>=57.6951)&(df_samp.latitude<=57.6953)]


    # In[89]:


    print(df_inc_s1.speedOverGround.max())
    print(df_dec_s1.speedOverGround.min())

    print(sog_max_limit)


    # ### Calculate the total distance and time

    # In[90]:


    df_upsog_fxcog_s1.head()


    # In[91]:


    dfloc.beta[dfloc.beta < 0]=dfloc.beta+360

    df_locs=dfloc[['beta', 'heading', 'latitude', 'longitude', 'Lat_inc', 'Lon_inc', 'Lat_dec', 'Lon_dec']]
    df_locs=df_locs.resample(tempo_res).mean()

    df_locs.shape


    # In[92]:


    df_info_tem={}
    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']    
    for i in sog:
        for j in cog:

            df_info_tem['fxcog_latitude']=df_locs.latitude.values
            df_info_tem['upcog_latitude']=df_locs.Lat_inc.values
            df_info_tem['dncog_latitude']=df_locs.Lat_dec.values
            df_info_tem['fxcog_longitude']=df_locs.longitude.values
            df_info_tem['upcog_longitude']=df_locs.Lon_inc.values
            df_info_tem['dncog_longitude']=df_locs.Lon_dec.values
            df_info_tem['heading']=df_locs.heading.values
            df_info_tem['beta']=df_locs.beta.values

            df_info_tem[i+'_'+j+'_'+'fuel']=df_Rt_Res[i+'_'+j].values
            df_info_tem[i+'_'+j+'_'+'distance']=0
            df_info_tem[i+'_'+j+'_'+'time']=0
            df_info_tem[i+'_'+j+'_'+'sog']=eval(i+'_'+j).speedOverGround
            df_info_tem[i+'_'+j+'_'+'cog']=eval(i+'_'+j).courseOverGroundTrue
            df_info_tem[i+'_'+j+'_'+'spapp']=eval(i+'_'+j).speedApparent
            df_info_tem[i+'_'+j+'_'+'wcdiff']=eval(i+'_'+j).wind_course_diff


    df_info=pd.DataFrame.from_dict(df_info_tem)
    df_info.shape


    # In[93]:


    df_info.head()


    # In[94]:


    dfst=pd.DataFrame([])
    dfst.index=['fuel', 'time', 'distance']
    # dfst[i+'_'+j].iloc['fuel']=40
    dfst[i+'_'+j]=40
    dfst[i+'_'+j].loc['fuel']=40


    # In[95]:


    R_rt={}; R_t={}; R_d={}; R_f={}; 

    dist=[]
    tmsan=[]
    fuelcns=[]
    df_st=pd.DataFrame([])
    df_stats=pd.DataFrame([])
    df_stats.index=['fuel', 'time', 'distance', 'Time_ExPorts']

    dfst_fuel=pd.DataFrame([])
    dfst_distance=pd.DataFrame([])
    dfst_time=pd.DataFrame([])

    dfcor=df_info


    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']   

    # sog=['df_fxsog']
    # cog=['fxcog_s1']   
    for i in sog:
        for j in cog:

            if j=='fxcog_s1':
                dfcor['lat1']=dfcor['fxcog_latitude']
                dfcor['lat2']=dfcor['fxcog_latitude'].shift(-1)
                dfcor['lon1']=dfcor['fxcog_longitude']
                dfcor['lon2']=dfcor['fxcog_longitude'].shift(-1)
            if j=='upcog_s1':
                dfcor['lat1']=dfcor['upcog_latitude']
                dfcor['lat2']=dfcor['upcog_latitude'].shift(-1)
                dfcor['lon1']=dfcor['upcog_longitude']
                dfcor['lon2']=dfcor['upcog_longitude'].shift(-1)
            if j=='dncog_s1':
                dfcor['lat1']=dfcor['dncog_latitude']
                dfcor['lat2']=dfcor['dncog_latitude'].shift(-1)
                dfcor['lon1']=dfcor['dncog_longitude']
                dfcor['lon2']=dfcor['dncog_longitude'].shift(-1)

            dfcor['dfr1']=dfcor[i+'_'+j+'_'+'fuel']
            dfcor = dfcor.reset_index(drop=True)
            dfcr=dfcor[['lat1', 'lat2', 'lon1', 'lon2']]
            dfcr.iloc[-1]=dfcr.iloc[-2] 
            Radious = 6373.0
            lat1=np.radians(dfcr['lat1'])
            lon1=np.radians(dfcr['lon1'])
            lat2=np.radians(dfcr['lat2'])
            lon2=np.radians(dfcr['lon2'])
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            distance = Radious * c

            if tempo_res=='1min':
                d_feulcns=dfcor['dfr1']/(60)
            if tempo_res=='1S':
                d_feulcns=dfcor['dfr1']/(60*60)
            if tempo_res=='10S':
                d_feulcns=dfcor['dfr1']/(60*60)*10
            if tempo_res=='30S':
                d_feulcns=dfcor['dfr1']/(60*60)*30



            time_i=distance/dfcor[i+'_'+j+'_'+'sog']*1000/60
            time_i=time_i.replace(np.inf,0)
            time_i=time_i.replace(np.nan,0)
            if time_sogfx==1:
                time_i[time_i>tim_tsh]=0

            df_info[i+'_'+j+'_'+'fuel']=dfcor['dfr1'].values
            df_info[i+'_'+j+'_'+'distance']=distance.values
            df_info[i+'_'+j+'_'+'time']=time_i.values


            R_t=time_i.sum()
            R_d=distance.sum()
            R_f=d_feulcns.sum()
            df_st['date']=df_R.index
            df_st[i+'_'+j+'_'+'fuel']=R_f
            df_st[i+'_'+j+'_'+'time']=R_t
            df_st[i+'_'+j+'_'+'distance']=R_d

            df_stats[i+'_'+j]=np.nan

            df_stats[i+'_'+j].loc['fuel']=R_f
            df_stats[i+'_'+j].loc['time']=R_t
            df_stats[i+'_'+j].loc['distance']=R_d
            df_stats[i+'_'+j].loc['Time_ExPorts']=0




    # df_info
    df_st=df_st.set_index(['date'])
    # df_st.T

    ### time_df_rt=df_Raw_RT.index[-1]-df_Raw_RT.index[0]
    ### time_df_rt=pd.to_timedelta(time_df_rt).total_seconds()/60

    df_stats

    # df_stats.shape


    # In[96]:


    df_comb_up_dn_fx=df_stats


    # In[97]:


    if plot_clear_fg==1:
        import matplotlib.pyplot as plt
        df_R = df_sim_dist_fuel

        # fig = plt.figure(figsize=(30,18))
        plt.rcParams["figure.figsize"] = (10, 5)

        df_R.plot.bar(rot=75, zorder=3)
        plt.title('Fuel', fontsize=18)
        plt.ylabel('Fuel Consumption (L)', fontsize=15, labelpad=10)
        plt.xlabel('', fontsize=15, labelpad=10)
        plt.yticks(fontsize=12);
        plt.xticks(fontsize=12, rotation = 0);
        plt.title('Fuel', fontsize=14) 
        plt.legend(bbox_to_anchor = (0.99, 1))
        plt.grid()

        plt.tight_layout()

        # fig.savefig('/content/gdrive/MyDrive/Investig_Hists_Routes/output2/Sorted_Hist_'+leg_name+'_'+day_start+'_'+day_end+'.png', dpi=100)

        plt.show();


    # In[98]:


        fig = plt.figure(figsize=(18,8))
        # plt.rcParams["figure.figsize"] = (10, 5)

        plt.subplot(1,3, 1)


        df_stats.loc['fuel'].plot.bar(rot=75, zorder=3)
        plt.title('Stats', fontsize=18)
        plt.ylabel('', fontsize=15, labelpad=10)
        plt.xlabel('', fontsize=15, labelpad=10)
        plt.yticks(fontsize=12);
        plt.xticks(fontsize=12, rotation = 70);
        plt.title('Stats', fontsize=14) 
        plt.legend(bbox_to_anchor = (0.99, 1))
        plt.grid()

        plt.subplot(1,3, 2)

        df_stats.loc['time'].plot.bar(rot=75, zorder=3)
        plt.title('Stats', fontsize=18)
        plt.ylabel('', fontsize=15, labelpad=10)
        plt.xlabel('', fontsize=15, labelpad=10)
        plt.yticks(fontsize=12);
        plt.xticks(fontsize=12, rotation = 70);
        plt.title('Stats', fontsize=14) 
        plt.legend(bbox_to_anchor = (0.99, 1))
        plt.grid()

        plt.subplot(1,3, 3)

        df_stats.loc['distance'].plot.bar(rot=75, zorder=3)
        plt.title('Stats', fontsize=18)
        plt.ylabel('', fontsize=15, labelpad=10)
        plt.xlabel('', fontsize=15, labelpad=10)
        plt.yticks(fontsize=12);
        plt.xticks(fontsize=12, rotation = 70);
        plt.title('Stats', fontsize=14) 
        plt.legend(bbox_to_anchor = (0.99, 1))
        plt.grid()

        plt.tight_layout()

        # fig.savefig('/content/gdrive/MyDrive/Investig_Hists_Routes/output2/Sorted_Hist_'+leg_name+'_'+day_start+'_'+day_end+'.png', dpi=100)

        plt.show();


        # In[99]:


        plt.rcParams["figure.figsize"] = (12, 6)


        # ax = df_stats.T.plot(kind='bar', secondary_y=['time'])
        # ax.set_ylabel('Fuel and Distance',fontsize=15, labelpad=10)

        ax = df_stats.T[['fuel', 'time']].plot(kind='bar', secondary_y=['time'])
        ax.set_ylabel('Fuel',fontsize=15, labelpad=10)
        ax.right_ax.set_ylabel('time (minutes)',fontsize=15, labelpad=10)
        # ax.set_title('Stats', fontsize=18)

        # ax.set_xlabel('', fontsize=16, labelpad=14)
        # ax.set_xticks(fontsize=12, rotation = 90)

        ax.grid(True)
        ax.set_axisbelow(True)

        # ax.set_xticks()
        # ax.legend()
        plt.show()



        # fig.savefig('/content/gdrive/MyDrive/Investig_Hists_Routes/output2/Sorted_Hist_'+leg_name+'_'+day_start+'_'+day_end+'.png', dpi=100)

        plt.show();


    # In[100]:
    if plot_clear_fg==1:

        import matplotlib.pyplot as plt

        df_R = df_sim_dist_fuel

        # fig = plt.figure(figsize=(30,18))
        plt.rcParams["figure.figsize"] = (10, 5)


        df_stats.plot.bar(rot=75, zorder=3)
        plt.title('Stats', fontsize=18)
        plt.ylabel('', fontsize=15, labelpad=10)
        plt.xlabel('', fontsize=15, labelpad=10)
        plt.yticks(fontsize=12);
        plt.xticks(fontsize=12, rotation = 0);
        plt.title('Stats', fontsize=14) 
        plt.legend(bbox_to_anchor = (0.99, 1))
        plt.grid()



        plt.tight_layout()

        # fig.savefig('/content/gdrive/MyDrive/Investig_Hists_Routes/output2/Sorted_Hist_'+leg_name+'_'+day_start+'_'+day_end+'.png', dpi=100)

        plt.show();


    # In[ ]:





    # ### =====================================================================================
    # 
    # ### Searching for the Best Combination of speedOverGround and CourseOverGround for Every Step on the Route

    # In[101]:


    df_Raw_RT.shape


    # In[102]:


    df_R.index


    # In[103]:


    df_locs.shape


    # In[104]:


    3**2*30


    # In[105]:


    df_sel_inps=df_Raw_RT[['speedOverGround', 'courseOverGroundTrue','wind_course_diff', 'windSpeedOverGround',
                           'directionTrue', 'distanceFromDock', 'distanceToNextDock','speedApparent']]
    df_Rt_Res.index=df_Raw_RT.index
    df_Rt_Res.Meas_fuel

    df_all_legs_comb=pd.concat([df_Rt_Res.Meas_fuel, df_sel_inps, df_info], axis=1)


    # In[106]:


    # df_Rt_Res.df_upsog_fxcog_s1
    # df_all_legs_comb.df_upsog_fxcog_s1_fuel


    # In[107]:


    df_all_legs_comb.shape
    # df_all_legs_comb
    # df_all_legs_comb.columns


    # In[108]:


    # stop=stop


    # In[109]:


    df_rt_sel_combs=pd.DataFrame([])
    dic_df_all={}
    dic_fuel={}; dic_distance={}; dic_time={}; dic_sog={}; dic_cog={}; dic_spapp={}; dic_wcdiff={}
    dic_heading={}; dic_beta={}; dic_lat={}; dic_lon={}
    dic_tmp={}
    sog=['df_upsog', 'df_dnsog', 'df_fxsog']
    cog=['fxcog_s1', 'upcog_s1', 'dncog_s1']   

    for R in range(len(df_all_legs_comb)):
    # for R in [10]:
    # for R in [12]:
        for i in sog:
            for j in cog:
                dic_fuel[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'fuel'].iloc[R]
                dic_distance[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'distance'].iloc[R]
                dic_time[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'time'].iloc[R]
                dic_sog[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'sog'].iloc[R]
                dic_cog[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'cog'].iloc[R]
                dic_spapp[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'spapp'].iloc[R]
                dic_wcdiff[i+'_'+j]=df_all_legs_comb[i+'_'+j+'_'+'wcdiff'].iloc[R]
                dic_heading[i+'_'+j]=df_all_legs_comb['heading'].iloc[R]
                dic_beta[i+'_'+j]=df_all_legs_comb['beta'].iloc[R]
                if j=='fxcog_s1':
                    dic_lat[i+'_'+j]=df_all_legs_comb['fxcog_latitude'].iloc[R]
                    dic_lon[i+'_'+j]=df_all_legs_comb['fxcog_longitude'].iloc[R]
                if j=='upcog_s1':
                    dic_lat[i+'_'+j]=df_all_legs_comb['upcog_latitude'].iloc[R]
                    dic_lon[i+'_'+j]=df_all_legs_comb['upcog_longitude'].iloc[R]
                if j=='dncog_s1':
                    dic_lat[i+'_'+j]=df_all_legs_comb['dncog_latitude'].iloc[R]
                    dic_lon[i+'_'+j]=df_all_legs_comb['dncog_longitude'].iloc[R]

        df_tmp=pd.DataFrame(dic_fuel, index=['fuel'])
        df_tmp=df_tmp.append(pd.DataFrame(dic_distance,index=['distance']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_time, index=['time']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_sog, index=['sog']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_cog, index=['cog']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_spapp, index=['spapp']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_wcdiff, index=['wcdiff']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_heading, index=['heading']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_beta, index=['direction']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_lat, index=['cog_latitude']))
        df_tmp=df_tmp.append(pd.DataFrame(dic_lon, index=['cog_longitude']))


        dic_df_all[R]=df_tmp

        if time_limit_fg==1:
            time_up=df_tmp.df_fxsog_fxcog_s1.loc['time']+df_tmp.loc['time'].std()
            time_lw=df_tmp.df_fxsog_fxcog_s1.loc['time']-df_tmp.loc['time'].std()
        if time_limit_fg==2:
            time_up=df_tmp.df_fxsog_fxcog_s1.loc['time']
            time_lw=df_tmp.df_fxsog_fxcog_s1.loc['time']-df_tmp.loc['time'].std()
        if time_limit_fg==3:
            tm_fx=df_tmp.df_fxsog_fxcog_s1.loc['time']
            tm_min=df_tmp.loc['time'].min()
            tm_q1=df_tmp.loc['time'].quantile(0.25)
            sog_avg=df_tmp.loc['sog'].mean()
            if (tm_fx > 15) & (sog_avg < 1) & ((tm_min==0) | (tm_q1==0)):
                time_up=tm_min
                time_lw=tm_min
                df_tmp.df_fxsog_fxcog_s1.loc['time']=0
            else:
                time_up=df_tmp.df_fxsog_fxcog_s1.loc['time']
                time_lw=df_tmp.df_fxsog_fxcog_s1.loc['time']-df_tmp.loc['time'].std()

        fuel_up=df_tmp.df_fxsog_fxcog_s1.loc['fuel']+df_tmp.loc['fuel'].std()
        fuel_lw=df_tmp.df_fxsog_fxcog_s1.loc['fuel']-df_tmp.loc['fuel'].std()*2

        time_sel=df_tmp.loc['time'][(df_tmp.loc['time']<time_up) & (df_tmp.loc['time']>time_lw)]

        fuel_sel=df_tmp[time_sel.index].loc['fuel'][(df_tmp.loc['fuel']<fuel_up) & (df_tmp.loc['fuel']>fuel_lw)]

        fuel_sel_min=fuel_sel[fuel_sel==fuel_sel.min()]
        if len(fuel_sel_min)==0:
            df_rt_sel_combs=df_rt_sel_combs.append(df_tmp['df_fxsog_fxcog_s1'].T)
        if len(fuel_sel_min.index)>=1:
            df_rt_sel_combs=df_rt_sel_combs.append(df_tmp[fuel_sel_min.index[0]].T)



    # df_tmp
    # df_tmp.T.describe()
    # dic_df_all

#     df_rt_sel_combs


    # In[110]:


    df_rt_sel_combs.shape


    # In[111]:


    df_rt_sel_combs['sog_cog_combs'] = df_rt_sel_combs.index
    first_column = df_rt_sel_combs.pop('sog_cog_combs')
    df_rt_sel_combs.insert(0, 'sog_cog_combs', first_column)
    df_rt_sel_combs.index=df_Raw_RT.index

    df_rt_sel_combs.head()


    # In[112]:


    from collections import Counter
    dist = Counter(df_rt_sel_combs["sog_cog_combs"].explode())
    dist_sel = pd.DataFrame.from_dict(dist, orient='index').reset_index()
    dist_sel= dist_sel.rename(columns={'index':'', 0:'sel_comb_count'})
    dist_sel=dist_sel.set_index(dist_sel[''])
    dist_sel.drop([''], axis=1, inplace=True)
    dist_sel=dist_sel.T
#     dist_sel
    df_comb_up_dn_fx=pd.concat([df_comb_up_dn_fx,dist_sel])
    df_comb_up_dn_fx=df_comb_up_dn_fx.replace(np.nan, 0)
#     df_comb_up_dn_fx


    # In[113]:


    # dic_lat  ####lat&lon change but with very small difference of fx, up, dn cases (i.e., see the std on different waypoints)
    # dic_lon

    # df_tmp.T.describe()
    # # df_tmp
    # dic_df_all


    # #### Calculate the fuel, speed and time for the initial route
    # df_Raw_RT

    # In[114]:


    # st=stop
    df_Raw_RT.shape


    # In[115]:


    R_rt={}; R_t={}; R_d={}; R_f={}; 

    dfst_time_ExPorts=[]
    dist=[]
    tmsan=[]
    fuelcns=[]
    df_st=pd.DataFrame([])
    df_st_rt=pd.DataFrame([])
    df_st_rt.index=['fuel', 'time', 'distance', 'Time_ExPorts']

    dfst_fuel=pd.DataFrame([])
    dfst_distance=pd.DataFrame([])
    dfst_time=pd.DataFrame([])
    df_info=pd.DataFrame([])


    if Exclud_port==0:
        dfcor=df_Raw_RT
    if Exclud_port==1:
        dfcor=df_Raw_RT[~(df_Raw_RT.speedOverGround<0.5) & ~(df_Raw_RT.deltaDistance<0.1)]

    dfcor['lat1']=dfcor['latitude']
    dfcor['lat2']=dfcor['latitude'].shift(-1)
    dfcor['lon1']=dfcor['longitude']
    dfcor['lon2']=dfcor['longitude'].shift(-1)


    dfcor['dfr1']=dfcor['engineFuelRate']
    dfcor = dfcor.reset_index(drop=True)
    dfcr=dfcor[['lat1', 'lat2', 'lon1', 'lon2']]
    dfcr.iloc[-1]=dfcr.iloc[-2] 
    Radious = 6373.0
    lat1=np.radians(dfcr['lat1'])
    lon1=np.radians(dfcr['lon1'])
    lat2=np.radians(dfcr['lat2'])
    lon2=np.radians(dfcr['lon2'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = Radious * c

    if tempo_res=='1min':
        d_feulcns=dfcor['dfr1']/(60)
    if tempo_res=='1S':
        d_feulcns=dfcor['dfr1']/(60*60)
    if tempo_res=='10S':
        d_feulcns=dfcor['dfr1']/(60*60)*10
    if tempo_res=='30S':
        d_feulcns=dfcor['dfr1']/(60*60)*30

    if tempo_res=='1min':
        dfst_time_ExPorts.append(len(dfcor))
    if tempo_res=='1S':
        dfst_time_ExPorts.append(len(dfcor))/60
    if tempo_res=='10S':
        ddfst_time_ExPorts.append(len(dfcor))/60*10
    if tempo_res=='30S':
        dfst_time_ExPorts.append(len(dfcor))/60*30

    time_i=distance/dfcor['speedOverGround']*1000/60
    time_i=time_i.replace(np.inf,0)
    time_i=time_i.replace(np.nan,0)

    df_info['fuel']=dfcor['dfr1'].values
    df_info['distance']=distance.values
    df_info['time']=time_i.values

    df_info['fuel']=dfcor['dfr1'].values
    df_info['distance']=distance.values
    df_info['time']=time_i.values

    R_t=time_i.sum()
    R_d=distance.sum()
    R_f=d_feulcns.sum()
    df_st['date']=df_R.index
    df_st['fuel']=R_f
    df_st['time']=R_t
    df_st['distance']=R_d
    df_st['Time_ExPorts']=dfst_time_ExPorts

    df_st_rt['meas_rt']=np.nan

    df_st_rt['meas_rt'].loc['fuel']=R_f
    df_st_rt['meas_rt'].loc['time']=R_t
    df_st_rt['meas_rt'].loc['distance']=R_d
    df_st_rt['meas_rt'].loc['Time_ExPorts']=dfst_time_ExPorts[0]



    # df_info
    df_st=df_st.set_index(['date'])
    # df_st.T

    df_st_rt

    # df_st_rt.shape


    # In[116]:


    R_rt={}; R_t={}; R_d={}; R_f={}; 

    dist=[]
    tmsan=[]
    fuelcns=[]
    df_st=pd.DataFrame([])
    df_stats=pd.DataFrame([])
    df_stats.index=['fuel', 'time', 'distance', 'Time_ExPorts']

    dfst_fuel=pd.DataFrame([])
    dfst_distance=pd.DataFrame([])
    dfst_time=pd.DataFrame([])
    df_info=pd.DataFrame([])


    # dfcor=df_rt_sel_combs
    if Exclud_port==0:
        dfcor=df_rt_sel_combs
    if Exclud_port==1:
        dfcor=df_rt_sel_combs[~(df_rt_sel_combs.sog<0.5) & ~(df_rt_sel_combs.distance<0.1)]

    dfcor['lat1']=dfcor['cog_latitude']
    dfcor['lat2']=dfcor['cog_latitude'].shift(-1)
    dfcor['lon1']=dfcor['cog_longitude']
    dfcor['lon2']=dfcor['cog_longitude'].shift(-1)


    dfcor['dfr1']=dfcor['fuel']
    dfcor = dfcor.reset_index(drop=True)
    dfcr=dfcor[['lat1', 'lat2', 'lon1', 'lon2']]
    dfcr.iloc[-1]=dfcr.iloc[-2] 
    Radious = 6373.0
    lat1=np.radians(dfcr['lat1'])
    lon1=np.radians(dfcr['lon1'])
    lat2=np.radians(dfcr['lat2'])
    lon2=np.radians(dfcr['lon2'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = Radious * c

    if tempo_res=='1min':
        d_feulcns=dfcor['dfr1']/(60)
    if tempo_res=='1S':
        d_feulcns=dfcor['dfr1']/(60*60)
    if tempo_res=='10S':
        d_feulcns=dfcor['dfr1']/(60*60)*10
    if tempo_res=='30S':
        d_feulcns=dfcor['dfr1']/(60*60)*30


    time_i=distance/dfcor['sog']*1000/60
    time_i=time_i.replace(np.inf,0)
    time_i=time_i.replace(np.nan,0)

    if time_sogfx==1:
        time_i[time_i>tim_tsh]=0

    df_info['fuel']=dfcor['dfr1'].values
    df_info['distance']=distance.values
    df_info['time']=time_i.values

    df_info['fuel']=dfcor['dfr1'].values
    df_info['distance']=distance.values
    df_info['time']=time_i.values

    R_t=time_i.sum()
    R_d=distance.sum()
    R_f=d_feulcns.sum()
    df_st['date']=df_R.index
    df_st['fuel']=R_f
    df_st['time']=R_t
    df_st['distance']=R_d

    df_stats['sel_comb']=np.nan

    df_stats['sel_comb'].loc['fuel']=R_f
    df_stats['sel_comb'].loc['time']=R_t
    df_stats['sel_comb'].loc['distance']=R_d
    df_stats['sel_comb'].loc['Time_ExPorts']=dfst_time_ExPorts[0]



    # df_info
    df_st=df_st.set_index(['date'])
    # df_st.T

    df_stats

    # df_stats.shape


    # In[117]:


    df_sel_combs=pd.concat([df_st_rt, df_stats, df_comb_up_dn_fx ],  axis=1)
    df_sel_combs.loc['Time_ExPorts']=df_sel_combs['meas_rt'].loc['Time_ExPorts']
    df_sel_combs=df_sel_combs.replace(np.nan, 0)
#     df_sel_combs


    # In[118]:


    # df_sim_dist_fuel


    # In[119]:

    if plot_clear_fg==1:
        plt.rcParams["figure.figsize"] = (12, 6)


        # ax = df_stats.T.plot(kind='bar', secondary_y=['time'])
        # ax.set_ylabel('Fuel and Distance',fontsize=15, labelpad=10)

        ax = df_sel_combs.T[['fuel', 'time']].plot(kind='bar', secondary_y=['time'])
        ax.set_ylabel('Fuel',fontsize=15, labelpad=10)
        ax.right_ax.set_ylabel('time (minutes)',fontsize=15, labelpad=10)
        # ax.set_title('Stats', fontsize=18)

        # ax.set_xlabel('', fontsize=16, labelpad=14)
        # ax.set_xticks(fontsize=12, rotation = 90)

        ax.grid(True)
        ax.set_axisbelow(True)

        # ax.set_xticks()
        # ax.legend()
        plt.show()

        # fig.savefig('/content/gdrive/MyDrive/Investig_Hists_Routes/output2/Sorted_Hist_'+leg_name+'_'+day_start+'_'+day_end+'.png', dpi=100)

        plt.show();


    # #### improvement of the route with selected combs of sog&cog over other routes

    # In[120]:


    # improve=(df_R['Meas_fuel'].values-df_R.values)/df_R['Meas_fuel'].values*100
    # improve=(df_sel_combs['df_fxsog_fxcog_s1'].loc['fuel']-df_sel_combs.loc['fuel'])/df_sel_combs['df_fxsog_fxcog_s1'].loc['fuel']*100
    improve=(df_sel_combs.loc['fuel']-df_sel_combs['sel_comb'].loc['fuel'])/df_sel_combs.loc['fuel']*100

    # print('improve of Pred fuel vs Meas_fuel',np.round(improve[0],2),'%')
    improvement=pd.DataFrame(data = improve).T
    improvement.columns=df_sel_combs.columns
    improvement.index=['improvement over initial route']


    # In[121]:


    df_improv=df_sel_combs
    df_improv=df_improv.append(improvement)
    # df_improv.index.tolist()
    # df_improv.index=['fuel consumption', 'improve over Meas_fuel']
    df_improv


    # ### Plot the route from the selected Comb of SOG and COG.

    # In[122]:


    print(df_rt_sel_combs.shape)
    df_rt_sel_combs.columns


    # In[123]:


    df=df_rt_sel_combs

    df=df.resample('1S').mean()
    # # df = df.dropna()
    # df=df.reset_index()
    # df.drop(['date', 'Unnamed: 0'], axis=1, inplace=True)

    df_filled1=df.interpolate(method='linear', limit_direction='forward', axis=0)
    df_filled2=df_filled1.interpolate(method='linear', limit_direction='backward', axis=0)

    df=df_filled2
    df.shape


    # In[124]:


    df['latitude']=df.cog_latitude
    df['longitude']=df.cog_longitude


    # In[125]:



    df.latitude.isna ().sum ()


    # In[126]:


    ###################################################
    # Define function to switch from lat/long to mercator coordinates
    def x_coord(x, y):

        lat = x
        lon = y

        r_major = 6378137.000
        x = r_major * np.radians(lon)
        scale = x/lon
        y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
            lat * (np.pi/180.0)/2.0)) * scale
        return (x, y)
    ###################################################
    output_notebook()

    # var2='fuel'
    var2='sog'

    dfloc=df

    dfloc['coordinates'] = list(zip(dfloc['latitude'], dfloc['longitude']))

    # Obtain list of mercator coordinates
    mercators = [x_coord(x, y) for x, y in dfloc['coordinates']]
    # Create mercator column in our df
    dfloc['mercator'] = mercators

    # Split that column out into two separate cols - mercator_x and mercator_y
    dfloc[['mercator_x', 'mercator_y']] = dfloc['mercator'].apply(pd.Series)
    ####################################################

    # Select tile set to use
    chosentile = get_provider(Vendors.STAMEN_TERRAIN)
    # chosentile = get_provider(Vendors.WIKIMEDIA)
    chosentile = get_provider(WIKIMEDIA)

    # Choose palette
    # palette = PRGn[11]
    # palette = PRGn[8]
    from bokeh.palettes import Plasma256
    palette=Plasma256

    # Tell Bokeh to use df as the source of the data
    source = ColumnDataSource(data=dfloc)

    # Define color mapper - which column will define the colour of the data points
    # Define color mapper - which column will define the colour of the data points
    color_mapper = linear_cmap(field_name = var2, palette = palette, low = dfloc[var2].min(), high = dfloc[var2].max())
    color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                        formatter = NumeralTickFormatter(format='0.0[0000]'), 
                        label_standoff = 13, width=8, location=(0,0))

    q = figure(title ='Vessel Route: Grötö-Kalvsund-Framnäs-Öckero, Gothenburg, Sweden. '+'Vessel Speed', x_axis_type="mercator", y_axis_type="mercator", 
                x_axis_label = 'Longitude', y_axis_label = 'Latitude')
    q.add_layout(color_bar, 'right')



    # Add map tile
    q.add_tile(chosentile)
      #Defines color bar    
    q.circle(x = 'mercator_x', y = 'mercator_y', line_color=color_mapper, color = color_mapper, source=source, size=2, fill_color= color_mapper, fill_alpha =0.7)

    q.add_tile(chosentile)



    # # #Save as HTML
    # output_file('/content/gdrive/MyDrive/Colab Notebooks/BokehAvocado-master/output/'+str(R_sz)+'_Sel_Routes_'+var2+'_Plot.html', title='Ferry Route in Gothenburg')

    if plot_clear_fg==1:
        show(q)


    # In[127]:


    print(df.shape)
    # df.columns

###################################################################
####################################################################

    dfst_start.append(df_Raw_RT.index[0])
    dfst_end.append(df_Raw_RT.index[-1]) 
    time_df_rt=df_Raw_RT.index[-1]-df_Raw_RT.index[0]
    
#     dfst_duration.append(pd.to_timedelta(time_df_rt).total_seconds()/60)

    if Exclud_port==0:
        df_Raw_RT_exclud=df_Raw_RT
    if Exclud_port==1:
        df_Raw_RT_exclud=df_Raw_RT[~(df_Raw_RT.speedOverGround<0.5) & ~(df_Raw_RT.deltaDistance<0.1)]
        
    if tempo_res=='1min':
        dfst_time_ExPorts_all.append(len(df_Raw_RT_exclud))
    if tempo_res=='1S':
        dfst_time_ExPorts_all.append(len(df_Raw_RT_exclud))/60
    if tempo_res=='10S':
        dfst_time_ExPorts_all.append(len(df_Raw_RT_exclud))/60*10
    if tempo_res=='30S':
        dfst_time_ExPorts_all.append(len(df_Raw_RT_exclud))/60*30
    
    dfst_fuel_sel.append(df_improv.loc['fuel'].sel_comb)
    dfst_fuel_meas.append(df_improv.loc['fuel'].meas_rt)
    dfst_fuel_fx.append(df_improv.loc['fuel'].df_fxsog_fxcog_s1)
    dfst_time_sel.append(df_improv.loc['time'].sel_comb)
    dfst_time_meas.append(df_improv.loc['time'].meas_rt)
    dfst_time_fx.append(df_improv.loc['time'].df_fxsog_fxcog_s1)
    dfst_distance_sel.append(df_improv.loc['distance'].sel_comb)
    dfst_distance_meas.append(df_improv.loc['distance'].meas_rt)
    dfst_distance_fx.append(df_improv.loc['distance'].df_fxsog_fxcog_s1)
    dfst_improve_sel.append(df_improv.loc['improvement over initial route'].sel_comb)
    dfst_improve_meas.append(df_improv.loc['improvement over initial route'].meas_rt)
    dfst_improve_fx.append(df_improv.loc['improvement over initial route'].df_fxsog_fxcog_s1)
    dfst_RTid.append(df_ock_gr['Unnamed: 0'][kk])
    
    df_RT_Stats_improve=pd.DataFrame({'RT_id':dfst_RTid, 'start_time':dfst_start, 'end_time':dfst_end, 'Time_ExPorts':dfst_time_ExPorts_all,
                                     'fuel_sel':dfst_fuel_sel, 'fuel_meas':dfst_fuel_meas, 'fuel_fx':dfst_fuel_fx,
                                     'time_sel':dfst_time_sel, 'time_meas':dfst_time_meas, 'time_fx':dfst_time_fx,
                                    'distance_sel':dfst_distance_sel, 'distance_meas':dfst_distance_meas,  'distance_fx':dfst_distance_fx,
                                     'improve_sel':dfst_improve_sel, 'improve_meas':dfst_improve_meas, 'improve_fx':dfst_improve_fx})
    
    
    df_RT_Si_tmp=pd.Series.to_frame(df_improv.loc['sel_comb_count'][2:]).T
    df_RT_Si_tmp.columns='count_'+df_RT_Si_tmp.columns
    df_RT_Si_tmp.index=list([kk])
    dfst_combs_dist=dfst_combs_dist.append(df_RT_Si_tmp)
    
    df_RT_Stats_improve=pd.concat([df_RT_Stats_improve, dfst_combs_dist], axis=1)

    df_RT_Stats_improve.to_excel('C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/Sensitivity_Analysis/output/All2020_Top_Bottom_RTs_Phi_Sens_Ock_Gr_'+str(ock_gr_fg)+'_top_btm_'+str(rt_tp_cases_fg)+'_Phi_'+str(Phi_course_fg)+'_tm_'+str(tempo_res)+'.xlsx')



Loading BokehJS ...

(1801, 43)


Loading BokehJS ...

5.9594000000000005
0.0
5.9594000000000005
(31, 12)


Loading BokehJS ...

...

(1801, 43)


Loading BokehJS ...

5.632625695304569
0.0
5.9594000000000005
(31, 12)


Loading BokehJS ...

(1801, 17)


In [ ]:
kk
# df_RT_Si_tmp.index=list([kk])
# df_RT_Stats_improve

In [ ]:
from IPython.display import Javascript
from nbconvert import HTMLExporter

def save_notebook():
    display(
        Javascript("IPython.notebook.save_notebook()"),
        include=['application/javascript']
    )

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)


# In[ ]:


cwd = os.getcwd() 
directory = os.path.join('C:/Users/'+user_name+'/iHelm/Sensitivity_Analysis')

os.chdir(directory) 
####################

impfl_id=str(round(improve[0].mean(),2))

import time

save_notebook()
time.sleep(3)
current_file = 'All_Top_Bottom_RTs_Phi_Searching_Sensitivity_ANN.ipynb'
output_file = 'C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/Pred_DT_Results/All_Top_Bottom_RTs_Sensitivity_'+tempo_res+'_'+impfl_id+'_crse_'+str(course_fg)+'_Phi_'+str(Phi_course_fg)+'_ock_gr_'+str(ock_gr_fg)+'_base_'+str(rt_base)+'_top_'+str(rt_tp_cases_fg)+'_tpbt_'+str(rt_tp)+'_Pred_Fuel.html'
output_file = 'C:/Users/'+user_name+'/OneDrive - Högskolan i Halmstad/iHelm/Sensitivity_Analysis/output/All_Top_Bottom_RTs_Sensitivity_'+tempo_res+'_'+impfl_id+'_crse_'+str(course_fg)+'_Phi_'+str(Phi_course_fg)+'_ock_gr_'+str(ock_gr_fg)+'_base_'+str(rt_base)+'_top_'+str(rt_tp_cases_fg)+'_tpbt_'+str(rt_tp)+'_Pred_Fuel.html'

# os.chdir('C:/Users/mohabu/OneDrive - Högskolan i Halmstad/Pred_DT_Results')
output_HTML(current_file, output_file)



In [ ]:
df_RT_Stats_improve 

In [ ]:
df[(df.index >= df_ock_gr.date_start.iloc[rt]) & (df.index <= df_ock_gr.date_end.iloc[rt])].head()

In [ ]:
kk

In [ ]:
df_ock_gr.date_start.iloc[kk]

In [ ]:
df_ock_gr.iloc[1]

In [ ]:
df_Raw_RT.shape

In [ ]:
df_improv

In [ ]:
df.shape

In [ ]:
# df_ins.head()
# df_ins.shape

In [ ]:
tempo_res

In [ ]:
std